In [1]:
#!pip install pandas

In [2]:
import pandas as pd
import time
from datetime import datetime, date
import glob
import re

In [3]:
#観測地点のデータ
url = 'https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/data-maxtemp/meta/points_list.csv'
points = pd.read_csv(url)

In [4]:
points.head()

,観測所番号,prec_no,国際地点番号,都道府県,name,地点,pref,capitol
0,11001,11,NaN,北海道宗谷地方,宗谷岬,宗谷岬（ソウヤミサキ）,北海道,NaN
1,11016,11,47401.0,北海道宗谷地方,稚内,稚内（ワッカナイ）,北海道,NaN
2,11046,11,NaN,北海道宗谷地方,礼文,礼文（レブン）,北海道,NaN
3,11061,11,NaN,北海道宗谷地方,声問,声問（コエトイ）,北海道,NaN
4,11076,11,NaN,北海道宗谷地方,浜鬼志別,浜鬼志別（ハマオニシベツ）,北海道,NaN


In [5]:
def get_historical_maxtemp(prec_no, md):
  #変数
  #全域は00
  block_no = '00'
  year =md.year
  month = md.month
  #データ取得
  url = f'https://www.data.jma.go.jp/stats/etrn/view/daily_h1.php?prec_no={prec_no}&block_no={block_no}&year={year}&month={str(month).zfill(2)}&day=&view=p3'
  tables = pd.read_html(url)
  df = tables[0]
  df.columns = df.columns.str.replace("*", "", regex=False)
  df['date'] =md
  df['date'] = df.apply(lambda row: row['date'].replace(day=row['日']), axis=1)
  df = df.drop(['日'],axis=1).set_index('date')
  return df

取得済みのファイル

In [6]:
%pwd

'/Users/mio/Documents/GitHub/Nikkei-Visual&Data/Heatwave'

In [7]:
file_dir = "./data-maxtemp/timeseries-data-by-points/data-raw/"

In [8]:
file_list = glob.glob(f'{file_dir}prec-*/**/*.csv', recursive=True)

In [9]:
retrieved = pd.DataFrame(columns = ['prec_no','yyyymm','data'])

pattern = r"/prec-(\d+)/jma-maxtemp-hs-\d+-(\d+)\.csv$"

for f in file_list:
    prec, yyyymm = re.search(pattern, f).groups()
    data =  {'prec_no': [int(prec)], 'yyyymm':[yyyymm],'data':[1]}
    retrieved = pd.concat([retrieved, pd.DataFrame(data)])

In [10]:
retrieved.head()

,prec_no,yyyymm,data
0,14,200112,1
0,14,200106,1
0,14,200310,1
0,14,200304,1
0,14,200502,1


期間

In [11]:
dates = pd.date_range('2000-01-01', '2023-07-01', freq='M')

In [12]:
errors = []
for prec in points.prec_no.unique():
  points_n = points[points.prec_no==prec]
  points_n_dic = points_n.set_index('name')['観測所番号'].to_dict()

  for md in dates:
    #try:
    if 0 ==0:
      if retrieved[(retrieved.prec_no==prec)&(retrieved.yyyymm==md.strftime('%Y%m'))].shape[0] > 0:
        print('file already retrieved:',prec, md)
        pass
      else:
        by_prec = get_historical_maxtemp(prec, md)

        by_prec = by_prec.rename_axis('name', axis=1).unstack().rename('maxtemp').reset_index()
        by_prec['prec_no'] = prec
        by_prec['points_no'] = by_prec.name.map(points_n_dic)

        #output monthly data
        filename = f"jma-maxtemp-hs-{prec}-{md.strftime('%Y%m')}.csv"
        filepath = f"{file_dir}prec-{prec}/{filename}"
        by_prec.to_csv(filepath, index= False)
        #print result
        print('OK:',prec, md)
        #time.sleep(1)
    #except:
      #errors.append({'date':md, 'prec_no':prec})
      #print result
      #print('FAILED:',prec, md)

file already retrieved: 11 2000-01-31 00:00:00
file already retrieved: 11 2000-02-29 00:00:00
file already retrieved: 11 2000-03-31 00:00:00
file already retrieved: 11 2000-04-30 00:00:00
file already retrieved: 11 2000-05-31 00:00:00
file already retrieved: 11 2000-06-30 00:00:00
file already retrieved: 11 2000-07-31 00:00:00
file already retrieved: 11 2000-08-31 00:00:00
file already retrieved: 11 2000-09-30 00:00:00
file already retrieved: 11 2000-10-31 00:00:00
file already retrieved: 11 2000-11-30 00:00:00
file already retrieved: 11 2000-12-31 00:00:00
file already retrieved: 11 2001-01-31 00:00:00
file already retrieved: 11 2001-02-28 00:00:00
file already retrieved: 11 2001-03-31 00:00:00
file already retrieved: 11 2001-04-30 00:00:00
file already retrieved: 11 2001-05-31 00:00:00
file already retrieved: 11 2001-06-30 00:00:00
file already retrieved: 11 2001-07-31 00:00:00
file already retrieved: 11 2001-08-31 00:00:00
file already retrieved: 11 2001-09-30 00:00:00
file already 

file already retrieved: 11 2018-09-30 00:00:00
file already retrieved: 11 2018-10-31 00:00:00
file already retrieved: 11 2018-11-30 00:00:00
file already retrieved: 11 2018-12-31 00:00:00
file already retrieved: 11 2019-01-31 00:00:00
file already retrieved: 11 2019-02-28 00:00:00
file already retrieved: 11 2019-03-31 00:00:00
file already retrieved: 11 2019-04-30 00:00:00
file already retrieved: 11 2019-05-31 00:00:00
file already retrieved: 11 2019-06-30 00:00:00
file already retrieved: 11 2019-07-31 00:00:00
file already retrieved: 11 2019-08-31 00:00:00
file already retrieved: 11 2019-09-30 00:00:00
file already retrieved: 11 2019-10-31 00:00:00
file already retrieved: 11 2019-11-30 00:00:00
file already retrieved: 11 2019-12-31 00:00:00
file already retrieved: 11 2020-01-31 00:00:00
file already retrieved: 11 2020-02-29 00:00:00
file already retrieved: 11 2020-03-31 00:00:00
file already retrieved: 11 2020-04-30 00:00:00
file already retrieved: 11 2020-05-31 00:00:00
file already 

file already retrieved: 13 2014-02-28 00:00:00
file already retrieved: 13 2014-03-31 00:00:00
file already retrieved: 13 2014-04-30 00:00:00
file already retrieved: 13 2014-05-31 00:00:00
file already retrieved: 13 2014-06-30 00:00:00
file already retrieved: 13 2014-07-31 00:00:00
file already retrieved: 13 2014-08-31 00:00:00
file already retrieved: 13 2014-09-30 00:00:00
file already retrieved: 13 2014-10-31 00:00:00
file already retrieved: 13 2014-11-30 00:00:00
file already retrieved: 13 2014-12-31 00:00:00
file already retrieved: 13 2015-01-31 00:00:00
file already retrieved: 13 2015-02-28 00:00:00
file already retrieved: 13 2015-03-31 00:00:00
file already retrieved: 13 2015-04-30 00:00:00
file already retrieved: 13 2015-05-31 00:00:00
file already retrieved: 13 2015-06-30 00:00:00
file already retrieved: 13 2015-07-31 00:00:00
file already retrieved: 13 2015-08-31 00:00:00
file already retrieved: 13 2015-09-30 00:00:00
file already retrieved: 13 2015-10-31 00:00:00
file already 

file already retrieved: 14 2021-01-31 00:00:00
file already retrieved: 14 2021-02-28 00:00:00
file already retrieved: 14 2021-03-31 00:00:00
file already retrieved: 14 2021-04-30 00:00:00
file already retrieved: 14 2021-05-31 00:00:00
file already retrieved: 14 2021-06-30 00:00:00
file already retrieved: 14 2021-07-31 00:00:00
file already retrieved: 14 2021-08-31 00:00:00
file already retrieved: 14 2021-09-30 00:00:00
file already retrieved: 14 2021-10-31 00:00:00
file already retrieved: 14 2021-11-30 00:00:00
file already retrieved: 14 2021-12-31 00:00:00
file already retrieved: 14 2022-01-31 00:00:00
file already retrieved: 14 2022-02-28 00:00:00
file already retrieved: 14 2022-03-31 00:00:00
file already retrieved: 14 2022-04-30 00:00:00
file already retrieved: 14 2022-05-31 00:00:00
file already retrieved: 14 2022-06-30 00:00:00
file already retrieved: 14 2022-07-31 00:00:00
file already retrieved: 14 2022-08-31 00:00:00
file already retrieved: 14 2022-09-30 00:00:00
file already 

file already retrieved: 17 2011-12-31 00:00:00
file already retrieved: 17 2012-01-31 00:00:00
file already retrieved: 17 2012-02-29 00:00:00
file already retrieved: 17 2012-03-31 00:00:00
file already retrieved: 17 2012-04-30 00:00:00
file already retrieved: 17 2012-05-31 00:00:00
file already retrieved: 17 2012-06-30 00:00:00
file already retrieved: 17 2012-07-31 00:00:00
file already retrieved: 17 2012-08-31 00:00:00
file already retrieved: 17 2012-09-30 00:00:00
file already retrieved: 17 2012-10-31 00:00:00
file already retrieved: 17 2012-11-30 00:00:00
file already retrieved: 17 2012-12-31 00:00:00
file already retrieved: 17 2013-01-31 00:00:00
file already retrieved: 17 2013-02-28 00:00:00
file already retrieved: 17 2013-03-31 00:00:00
file already retrieved: 17 2013-04-30 00:00:00
file already retrieved: 17 2013-05-31 00:00:00
file already retrieved: 17 2013-06-30 00:00:00
file already retrieved: 17 2013-07-31 00:00:00
file already retrieved: 17 2013-08-31 00:00:00
file already 

OK: 18 2017-12-31 00:00:00
OK: 18 2018-01-31 00:00:00
OK: 18 2018-02-28 00:00:00
OK: 18 2018-03-31 00:00:00
OK: 18 2018-04-30 00:00:00
OK: 18 2018-05-31 00:00:00
OK: 18 2018-06-30 00:00:00
OK: 18 2018-07-31 00:00:00
OK: 18 2018-08-31 00:00:00
OK: 18 2018-09-30 00:00:00
OK: 18 2018-10-31 00:00:00
OK: 18 2018-11-30 00:00:00
OK: 18 2018-12-31 00:00:00
OK: 18 2019-01-31 00:00:00
OK: 18 2019-02-28 00:00:00
OK: 18 2019-03-31 00:00:00
OK: 18 2019-04-30 00:00:00
OK: 18 2019-05-31 00:00:00
OK: 18 2019-06-30 00:00:00
OK: 18 2019-07-31 00:00:00
OK: 18 2019-08-31 00:00:00
OK: 18 2019-09-30 00:00:00
OK: 18 2019-10-31 00:00:00
OK: 18 2019-11-30 00:00:00
OK: 18 2019-12-31 00:00:00
OK: 18 2020-01-31 00:00:00
OK: 18 2020-02-29 00:00:00
OK: 18 2020-03-31 00:00:00
OK: 18 2020-04-30 00:00:00
OK: 18 2020-05-31 00:00:00
OK: 18 2020-06-30 00:00:00
OK: 18 2020-07-31 00:00:00
OK: 18 2020-08-31 00:00:00
OK: 18 2020-09-30 00:00:00
OK: 18 2020-10-31 00:00:00
OK: 18 2020-11-30 00:00:00
OK: 18 2020-12-31 00:00:00
O

HTTPError: HTTP Error 504: Gateway Time-out